Script responsável por acessar a API do Cadastro Nacional de Usuários de Recursos Hídricos (CNARH) e obter os dados de usos outorgados no Estado de Goiás.

Outorgas estaduais superficiais = https://dadosabertos.ana.gov.br/datasets/3b1ed9295d2c428a8f0ec6e6ffcfeddf_1/explore

Outorgas estaduais subterrâneas = https://dadosabertos.ana.gov.br/datasets/096e3aae5bd24c759c3b08fcfa0e33c3_2/explore

Por meio do API explorer disponibilizado, filtra-se os dados para o Estado de Goiás, gerando uma URL para consulta. Essa URL retorna um JSON que posteriormente
é tratado e convertido em um formato vetorial (gpkg) para ser trabalhado em ambiente SIG. 

In [38]:
import requests 
import pandas as pd 
import geopandas as gpd 
from datetime import datetime

def get_cnarh40(url_superf,url_subt):
    superf_request = requests.get(url_superf)
    subt_request = requests.get(url_subt)
    now = datetime.now()

    if superf_request.status_code != 200 or subt_request.status_code != 200:
        return 'Erro na requisição. Verificar link da API'
    else:
        response_list_superf = superf_request.json() #lendo o resultado da requisição para superf
        response_list_subt = subt_request.json() #lendo o resultado da requisição para subt
        
        #transformando o json de superficial em um dataframe
        get_superf_features = response_list_superf['features']
        get_superf_attributes = [feature['attributes'] for feature in get_superf_features]
        cnarh_superf_df = pd.DataFrame(get_superf_attributes)

        #transformando o json de subterrânea em um dataframe
        get_subt_features = response_list_subt['features']
        get_subt_attributes = [feature['attributes'] for feature in get_subt_features]
        cnarh_subt_df = pd.DataFrame(get_subt_attributes)

        #unindo superficial e subterrânea em um único dataframe
        cnarh_superf_subt_union = pd.concat([cnarh_superf_df,cnarh_subt_df])

        #criando uma coluna com a data da request
        cnarh_superf_subt_union['api_query_date'] = now.strftime("%d/%m/%Y %H:%M:%S")
        #convertendo o dataframe em um geodataframe
        gdf_cnarh = gpd.GeoDataFrame(cnarh_superf_subt_union,geometry=gpd.points_from_xy(cnarh_superf_subt_union['INT_NU_LONGITUDE'],cnarh_superf_subt_union['INT_NU_LATITUDE']))

        #salvando o geodataframe em um formato vetorial (Geopackage)
        return gdf_cnarh.to_file('cnarh40_superf_subt.gpkg')

        
    

In [39]:
url_superf = "https://www.snirh.gov.br/arcgis/rest/services/SFI/cnarh_dados_abertos_v31082022/FeatureServer/1/query?where=ORG_UF%20%3D%20'GO'&outFields=*&outSR=4326&f=json"
url_subt = "https://www.snirh.gov.br/arcgis/rest/services/SFI/cnarh_dados_abertos_v31082022/FeatureServer/2/query?where=ORG_UF%20%3D%20'GO'&outFields=*&outSR=4326&f=json"

get_cnarh40(url_superf,url_subt)
